# Introduction to plotting skew-T, log p diagrams and hodographs in MetPy.
Now that you've gone to the trouble of plotting a skew-T by hand, let's make MetPy do the dirty work.

We previously used a package called <a href = "https://sharp.weather.ou.edu/dev/">SHARPPy</a>, which produces a single graphic containing both a skew-T, log p diagram and hodograph, with lots of convective indices attached. This graphic is intended for quick, at-a-glance assessment of the storm potential associated with an environmental sounding. This graphic is very busy, and would not be appropriate for use in a journal article. For finer control of the plots, we can use MetPy instead. 

It is assumed that the file <b>2021-10-11_1239.sharppy_mod.txt</b> (which is on Brightspace) lives in the same directory as this notebook.

In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd
# Huzzah! MetPy already has pre-defined Hodograph and Skew-T classes!
from metpy.plots import add_metpy_logo, Hodograph, SkewT
from metpy.units import units
import metpy.calc as mpcalc

## Recall that our raw sounding data (in abbreviated form) looks like this (with prepended line numbers):

0: `%TITLE%`<br>
1: ` XXX   211011/1744`<br>
2: ` Saved by user: User on 20211011/1837 UTC`<br>
3: `   LEVEL       HGHT       TEMP       DWPT       WDIR       WSPD`<br>
4: `-------------------------------------------------------------------`<br>
5: `%RAW%`<br>
6: `  986.60,    200.00,     25.00,     19.00,    125.00,      2.60`<br>
7:`  983.94,    209.00,     25.76,     17.90,    125.00,      2.60`<br>
8: `  981.72,    229.00,     25.37,     17.67,    166.64,     14.81`<br>
  ...<br>
-4: `  193.11,  12609.00,    -46.23,    -58.58,    217.21,     35.66`<br>
-3: `  192.54,  12629.00,    -46.23,    -58.64,    214.16,     37.03`<br>
-2: `  191.96,  12649.00,    -46.23,    -58.69,    212.05,     37.78`<br>
-1: `%END%`<br>

Unlike some other programming languages, Python **indexing starts at 0**. Additionally, an index of **-1** means, **"the last index"**. This is useful if, as sounding files often are, the length of the file varies from data set to data set (i.e., is unpredictable).

Looking at our sounding file, we find that lines 0, 1, 2, 4, 5, and -1 are formatted for human readability, rather than for formatted reading by a computer. Additionally, line 3 (LEVEL HGHT etc) contains the column headings. Pandas expects each line to have the same number of data entries (in this case, six). We can make the data import cleanly by passing a couple of extra arguments to pandas.read_csv():
1. `names = [A list of names]`: Python use the column labels specified in the list.
2. `skiprows = [A list of integers]`: Python will ignore the indicated rows when reading in the file.
3. `skipfooter = 1, engine = 'python'`: Skip the last line of the file (the one containing `%END%`, which we don't need). The 'engine' argument suppresses an annoying warning message.

## The code below reads in the sounding file and returns a pandas DataFrame object containing its contents.

In [5]:
sounding = pd.read_csv('2021-10-11_1239.sharppy_mod.txt', skiprows = [0, 1, 2, 3, 4, 5], 
                       names = ["Pressure", "Height", "Temperature", "Dewpoint", "Wind Direction", "Wind Speed"],
                      skipfooter = 1, engine = 'python')
sounding  # Print out the contents of the variable 'sounding' to the screen

FileNotFoundError: [Errno 2] No such file or directory: '2021-10-11_1239.sharppy_mod.txt'

## Next we need to massage the data to attach units, as MetPy requires.

In [ ]:
p = sounding['Pressure'].values * units('hectopascal')
z = sounding['Height'].values * units('meters')
t = sounding['Temperature'].values * units('degC')
d = sounding['Dewpoint'].values * units('degC')

# Now let's create the skew-T chart. 
Matplotlib requires that we create a figure and axes to draw this on. Thanks to the import statement way back up at the top, 'SkewT' is a class of object in our Python namespace. We create an instance called 'skew', then invoke several of its methods to create a familiar-looking skew-T chart.

In [ ]:
fig = plt.figure(figsize = (10, 10)) # Create a 10 inch-by-10 inch figure object.
add_metpy_logo(fig, 115, 100) # Acknowledge the nice MetPy team who made this package.
skew = SkewT(fig, rotation=45)

# Add the relevant special lines
skew.plot_dry_adiabats() # Light red
skew.plot_moist_adiabats() # Light blue
skew.plot_mixing_lines() # Light green
# Plot a zero degree isotherm
skew.ax.axvline(0, color='cyan', linestyle='--', linewidth=1.5)

### PLOT THE SOUNDING ###
skew.plot(p, t, 'darkred')
skew.plot(p, d, 'darkblue')
### END SOUNDING PLOTTING ###

plt.title('Purdue Sounding, 11 October 2021 1745 UTC')
plt.xlabel('Temperature (deg C)')
plt.ylabel('Pressure (hPa)')
plt.savefig('skew1.png')
plt.show()

# Now here's a skew-T, log p diagram we could put in a journal article. 
It's clean and informative. However, let's ask MetPy to do more. Specifically, it can calculate a parcel trajectory for us.

# Calculate the parcel profile.
This is just a series of temperatures in degrees Celsius.

In [ ]:
parcel_prof = mpcalc.parcel_profile(p, t[0], d[0]).to('degC')
parcel_prof # Print out the contents of the variable 'parcel_prof' to the screen

## MetPy can also estimate LCL, LFC, EL, CAPE, and CIN based upon this parcel profile.

In [ ]:
cape, cin = mpcalc.cape_cin(p, t, d, parcel_prof)
lcl = mpcalc.lcl(p[0], t[0], d[0]) # This is LCL based on the surface parcel.
lfc = mpcalc.lfc(p, t, d, parcel_prof, which = 'most_cape')
el = mpcalc.el(p, t, d, parcel_prof, which = 'most_cape')

print('CAPE = ', cape)
print('CIN = ', cin)
print('LCL = ', lcl)
print('LFC = ', lfc)
print('EL', el)

The LCL, LFC, and EL are formatted strangely because print() doesn't quite know how to handle quantities with units attached. However, we can easily understand them. 

# Let's redraw the skew-T, log p diagram, now with the parcel profile and these thermodynamic quantities added. 
We'll also shade regions of CAPE and CIN to boot.

In [ ]:
fig = plt.figure(figsize = (10, 10)) # Create a 10 inch-by-10 inch figure object.
add_metpy_logo(fig, 115, 100)
skew = SkewT(fig, rotation=45)

# Add the relevant special lines
skew.plot_dry_adiabats() # Light red
skew.plot_moist_adiabats() # Light blue
skew.plot_mixing_lines() # Light green
# Plot a zero degree isotherm
skew.ax.axvline(0, color='cyan', linestyle='--', linewidth=1.5)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot.
skew.plot(p, t, 'darkred')
skew.plot(p, d, 'darkblue')
### NEW STUFF ###
# Plot parcel profile
skew.plot(p, parcel_prof, 'black')
# Shade areas of CAPE and CIN
skew.shade_cin(p, t, parcel_prof, d) # Light blue shading
skew.shade_cape(p, t, parcel_prof) # Light red shading
### END OF NEW STUFF ###

plt.title('Purdue Sounding, 11 October 2021 1745 UTC')
plt.xlabel('Temperature (deg C)')
plt.ylabel('Pressure (hPa)')
plt.savefig('skew2.png')
plt.show()

# Let's move on to the hodograph.
## First, we have to convert the wind data from speed and direction to u- and v-components.

In [ ]:
wd = sounding['Wind Direction'].values * units('degrees')
ws = sounding['Wind Speed'].values * units('knots')

u, v = mpcalc.wind_components(ws, wd)
plt.plot(u, v)
# Create a basic plot just to ensure that the values look sane.
plt.show()

## To create a proper hodograph, let's instantiate an instance of the Hodograph class.

In [ ]:
fig = plt.figure()
hodo = Hodograph(component_range=80.)
hodo.add_grid(increment=20)
hodo.plot(u, v)   # Plain hodograph
plt.xlabel('East-west wind (kts)')
plt.ylabel('North-south wind (kts)')
plt.title('Purdue Hodograph, 11 October 2021 1745 UTC')
plt.savefig('hodo1.png')
plt.show()

## While this hodograph is complete, it's impossible to tell which point on the hodograph corresponds to a particular height. 

## We can color-code them by height, like so:

In [ ]:
fig = plt.figure()
hodo = Hodograph(component_range=80.)
hodo.add_grid(increment=20)
hodo.plot_colormapped(u, v, z)  # Plot a line colored by altitude
plt.xlabel('East-west wind (kts)')
plt.ylabel('North-south wind (kts)')
plt.title('Purdue Hodograph, 11 October 2021 1745 UTC')
plt.savefig('hodo2.png')
plt.show()

## If we want the hodograph to be a little less squiggly, we can tell MetPy to plot only every *n*th point. 
In this case, I'm only plotting every 25th point, or roughly every 400 m.

In [ ]:
fig = plt.figure()
hodo = Hodograph(component_range=80.)
hodo.add_grid(increment=20)
hodo.plot_colormapped(u[::25], v[::25], z[::25])  # Plot a line colored by altitude
plt.xlabel('East-west wind (kts)')
plt.ylabel('North-south wind (kts)')
plt.title('Purdue Hodograph, 11 October 2021 1745 UTC')
plt.savefig('hodo3.png')
plt.show()

# Lastly, just for kicks, let's plot the hodograph in the corner of our skew-T, log p plot. 
There's a lot of wasted space near the top of our skew-T because our sounding ends at about 200 mb. We can plot the hodograph as an inset.

In [ ]:
fig = plt.figure(figsize = (10, 10)) # Create a 10 inch-by-10 inch figure object.
add_metpy_logo(fig, 115, 100)
skew = SkewT(fig, rotation=45)

# Add the relevant special lines
skew.plot_dry_adiabats() # Light red
skew.plot_moist_adiabats() # Light blue
skew.plot_mixing_lines() # Light green
# Plot a zero degree isotherm
skew.ax.axvline(0, color='cyan', linestyle='--', linewidth=1.5)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot.
skew.plot(p, t, 'darkred')
skew.plot(p, d, 'darkblue')
# Plot parcel profile
skew.plot(p, parcel_prof, 'black')
# Shade areas of CAPE and CIN
skew.shade_cin(p, t, parcel_prof, d) # Light blue shading
skew.shade_cape(p, t, parcel_prof) # Light red shading

plt.title('Purdue Sounding, 11 October 2021 1745 UTC')
plt.xlabel('Temperature (deg C)')
plt.ylabel('Pressure (hPa)')

### HODOGRAPH INSET ###
# Create an inset axes object that is 30% width and height of the
# figure and put it in the upper right hand corner.
ax_hod = inset_axes(skew.ax, '30%', '30%', loc=1)
hodo = Hodograph(ax_hod, component_range=80.)
hodo.add_grid(increment=20)
hodo.plot_colormapped(u, v, z)  # Plot a line colored by height
plt.xlabel('u (kts)')  # I've shortened these labels because the inset is small.
plt.ylabel('v (kts)') 
### END OF HODOGRAPH INSET ###

plt.savefig('skew3.png')
plt.show()

# Such a graphic should, of course, be accompanied by a descriptive caption explaining what *all* the curves mean. e.g.,
**Fig. 1.** Skew-T, log p diagram of a radiosounding taken at West Lafayette, Indiana at 1745 UTC on 11 October 2021, including temperature (deg C; dark red solid line), dew point (deg C; dark blue solid line), and surface parcel trajectory (deg C; black solid line). The region shaded pink (blue) corresponds to CAPE (CIN). Dashed red, blue, and green lines correspond to dry adiabats, moist adiabats, and water vapor mixing ratio lines, respectively. Inset: Hodograph from the same sounding (in kts), color shading (indigo to yellow) corresponds to increasing height (0.2 km MSL to 12.6 km MSL). Dashed gray rings are drawn every 20 kts.